# Ragas Evaluation with Llama Stack - Demo [inline execution]

This notebook demonstrates how to use the Ragas out-of-tree provider.


## 1. Setup and Imports


In [1]:
# Install dev packages if not already installed
# !uv pip install -e ".[dev]"

from datetime import datetime

from llama_stack_client import LlamaStackClient
from rich.pretty import pprint

## 2. Llama Stack Client Setup

- Make sure we have an inference model (model_type='llm')
- Make sure we have an embedding model (model_type='embedding')


In [2]:
client = LlamaStackClient(base_url="http://localhost:8321")

models = client.models.list()
pprint(models)

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/models "HTTP/1.1 200 OK"


[
│   Model(
│   │   identifier='all-MiniLM-L6-v2',
│   │   metadata={'embedding_dimension': 384.0},
│   │   api_model_type='embedding',
│   │   provider_id='ollama',
│   │   type='model',
│   │   provider_resource_id='all-minilm:latest',
│   │   model_type='embedding'
│   ),
│   Model(
│   │   identifier='ollama/granite3.3:2b',
│   │   metadata={},
│   │   api_model_type='llm',
│   │   provider_id='ollama',
│   │   type='model',
│   │   provider_resource_id='granite3.3:2b',
│   │   model_type='llm'
│   ),
│   Model(
│   │   identifier='ollama/all-minilm:l6-v2',
│   │   metadata={'embedding_dimension': 384.0, 'context_length': 512.0},
│   │   api_model_type='embedding',
│   │   provider_id='ollama',
│   │   type='model',
│   │   provider_resource_id='all-minilm:l6-v2',
│   │   model_type='embedding'
│   ),
│   Model(
│   │   identifier='all-minilm',
│   │   metadata={'embedding_dimension': 384.0, 'context_length': 512.0},
│   │   api_model_type='embedding',
│   │   provider_id='ollama',
│   │   type='model',
│   │   provider_resource_id='all-minilm:l6-v2',
│   │   model_type='embedding'
│   ),
│   Model(
│   │   identifier='nomic-embed-text',
│   │   metadata={'embedding_dimension': 768.0, 'context_length': 8192.0},
│   │   api_model_type='embedding',
│   │   provider_id='ollama',
│   │   type='model',
│   │   provider_resource_id='nomic-embed-text:latest',
│   │   model_type='embedding'
│   )
]

## 4. Dataset Preparation

Create a sample RAG evaluation dataset. In a real scenario, you would load your own dataset.


In [3]:
# Sample Ragas evaluation dataset
evaluation_data = [
    {
        "user_input": "What is the capital of France?",
        "response": "The capital of France is Paris.",
        "retrieved_contexts": [
            "Paris is the capital and most populous city of France."
        ],
        "reference": "Paris",
    },
    {
        "user_input": "Who invented the telephone?",
        "response": "Alexander Graham Bell invented the telephone in 1876.",
        "retrieved_contexts": [
            "Alexander Graham Bell was a Scottish-American inventor who patented the first practical telephone."
        ],
        "reference": "Alexander Graham Bell",
    },
    {
        "user_input": "What is photosynthesis?",
        "response": "Photosynthesis is the process by which plants convert sunlight into energy.",
        "retrieved_contexts": [
            "Photosynthesis is a process used by plants to convert light energy into chemical energy."
        ],
        "reference": "Photosynthesis is the process by which plants and other organisms convert light energy into chemical energy.",
    },
]

## 5. Dataset Registration

Register the dataset with Llama Stack's Datasets API using the direct rows approach.


In [4]:
# Register the dataset
dataset_id = "ragas_demo_dataset_inline"

dataset_response = client.datasets.register(
    dataset_id=dataset_id,
    purpose="eval/question-answer",  # RAG evaluation purpose
    source={"type": "rows", "rows": evaluation_data},
    metadata={
        "provider_id": "localfs",  # seems there's a bug in datasets
        "description": "Sample RAG evaluation dataset for Ragas demo",
        "size": len(evaluation_data),
        "format": "ragas",
        "created_at": datetime.now().isoformat(),
    },
)
pprint(dataset_response)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/datasets "HTTP/1.1 200 OK"


DatasetRegisterResponse(
│   identifier='ragas_demo_dataset_inline',
│   metadata={
│   │   'provider_id': 'localfs',
│   │   'description': 'Sample RAG evaluation dataset for Ragas demo',
│   │   'size': 3.0,
│   │   'format': 'ragas',
│   │   'created_at': '2025-09-25T19:25:53.739691'
│   },
│   provider_id='localfs',
│   purpose='eval/question-answer',
│   source=SourceRowsDataSource(
│   │   rows=[
│   │   │   {
│   │   │   │   'user_input': 'What is the capital of France?',
│   │   │   │   'response': 'The capital of France is Paris.',
│   │   │   │   'retrieved_contexts': ['Paris is the capital and most populous city of France.'],
│   │   │   │   'reference': 'Paris'
│   │   │   },
│   │   │   {
│   │   │   │   'user_input': 'Who invented the telephone?',
│   │   │   │   'response': 'Alexander Graham Bell invented the telephone in 1876.',
│   │   │   │   'retrieved_contexts': [
│   │   │   │   │   'Alexander Graham Bell was a Scottish-American inventor who patented the first practical telephone.'
│   │   │   │   ],
│   │   │   │   'reference': 'Alexander Graham Bell'
│   │   │   },
│   │   │   {
│   │   │   │   'user_input': 'What is photosynthesis?',
│   │   │   │   'response': 'Photosynthesis is the process by which plants convert sunlight into energy.',
│   │   │   │   'retrieved_contexts': [
│   │   │   │   │   'Photosynthesis is a process used by plants to convert light energy into chemical energy.'
│   │   │   │   ],
│   │   │   │   'reference': 'Photosynthesis is the process by which plants and other organisms convert light energy into chemical energy.'
│   │   │   }
│   │   ],
│   │   type='rows'
│   ),
│   type='dataset',
│   provider_resource_id='ragas_demo_dataset_inline',
│   owner=None
)

## 6. Benchmark Registration

Register a benchmark that defines what metrics to use for evaluation.


In [5]:
benchmark_id = f"ragas_demo_benchmark_{datetime.now().strftime('%Y%m%d_%H%M%S')}"

ragas_metrics = [
    "answer_relevancy",  # How relevant is the answer to the question?
    # "context_precision",     # How precise are the retrieved contexts?
    # "faithfulness",          # How faithful is the answer to the contexts?
    # "context_recall",        # How much of the ground truth is covered by contexts?
    # "answer_correctness"  # How correct is the answer compared to ground truth?
]

benchmark_response = client.benchmarks.register(
    benchmark_id=benchmark_id,
    dataset_id=dataset_id,
    scoring_functions=ragas_metrics,
    provider_id="trustyai_ragas",
    # metadata={
    #     "provider": "ragas",
    #     "version": "1.0",
    #     "metrics_count": len(ragas_metrics),
    #     "created_at": datetime.now().isoformat()
    # }
)

pprint(benchmark_response)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/eval/benchmarks "HTTP/1.1 200 OK"


None

In [6]:
benchmarks = client.benchmarks.list()
pprint(benchmarks[-2:])

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/eval/benchmarks "HTTP/1.1 200 OK"


[
│   Benchmark(
│   │   dataset_id='ragas_demo_dataset_inline',
│   │   identifier='ragas_demo_benchmark_20250925_192402',
│   │   metadata={},
│   │   provider_id='trustyai_ragas',
│   │   scoring_functions=['answer_relevancy'],
│   │   type='benchmark',
│   │   provider_resource_id='ragas_demo_benchmark_20250925_192402'
│   ),
│   Benchmark(
│   │   dataset_id='ragas_demo_dataset_inline',
│   │   identifier='ragas_demo_benchmark_20250925_192553',
│   │   metadata={},
│   │   provider_id='trustyai_ragas',
│   │   scoring_functions=['answer_relevancy'],
│   │   type='benchmark',
│   │   provider_resource_id='ragas_demo_benchmark_20250925_192553'
│   )
]

## 7. Evaluation Execution

Run the evaluation using our Ragas out-of-tree provider.


In [7]:
# since we can't set the embedding model in the benchmark config,
# the embedding model is set in the distribution run.yaml file (all-MiniLM-L6-v2)

job = client.eval.run_eval(
    benchmark_id=benchmark_id,
    benchmark_config={
        "eval_candidate": {
            "type": "model",
            # "model": "meta-llama/Llama-3.2-3B-Instruct",
            "model": "ollama/granite3.3:2b",
            "sampling_params": {"temperature": 0.1, "max_tokens": 100},
        },
        "scoring_params": {},
    },
)
pprint(job)

INFO:httpx:HTTP Request: POST http://localhost:8321/v1/eval/benchmarks/ragas_demo_benchmark_20250925_192553/jobs "HTTP/1.1 200 OK"


Job(
│   job_id='1',
│   status='in_progress',
│   result=None,
│   eval_config={
│   │   'embedding_model': 'all-MiniLM-L6-v2',
│   │   'ragas_config': {
│   │   │   'batch_size': None,
│   │   │   'show_progress': True,
│   │   │   'raise_exceptions': True,
│   │   │   'experiment_name': None,
│   │   │   'column_map': None
│   │   }
│   }
)

## 8. Results Display


In [8]:
job = client.eval.jobs.status(benchmark_id=benchmark_id, job_id=job.job_id)
pprint(job)

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/eval/benchmarks/ragas_demo_benchmark_20250925_192553/jobs/1 "HTTP/1.1 200 OK"


Job(
│   job_id='1',
│   status='in_progress',
│   result=None,
│   eval_config={
│   │   'embedding_model': 'all-MiniLM-L6-v2',
│   │   'ragas_config': {
│   │   │   'batch_size': None,
│   │   │   'show_progress': True,
│   │   │   'raise_exceptions': True,
│   │   │   'experiment_name': None,
│   │   │   'column_map': None
│   │   }
│   }
)

In [11]:
# wait a bit for the job to complete
pprint(client.eval.jobs.status(benchmark_id=benchmark_id, job_id=job.job_id))

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/eval/benchmarks/ragas_demo_benchmark_20250925_192553/jobs/1 "HTTP/1.1 200 OK"


Job(
│   job_id='1',
│   status='completed',
│   result={
│   │   'generations': [
│   │   │   {
│   │   │   │   'user_input': 'What is the capital of France?',
│   │   │   │   'retrieved_contexts': ['Paris is the capital and most populous city of France.'],
│   │   │   │   'response': 'The capital of France is Paris.',
│   │   │   │   'reference': 'Paris'
│   │   │   },
│   │   │   {
│   │   │   │   'user_input': 'Who invented the telephone?',
│   │   │   │   'retrieved_contexts': [
│   │   │   │   │   'Alexander Graham Bell was a Scottish-American inventor who patented the first practical telephone.'
│   │   │   │   ],
│   │   │   │   'response': 'Alexander Graham Bell invented the telephone in 1876.',
│   │   │   │   'reference': 'Alexander Graham Bell'
│   │   │   },
│   │   │   {
│   │   │   │   'user_input': 'What is photosynthesis?',
│   │   │   │   'retrieved_contexts': [
│   │   │   │   │   'Photosynthesis is a process used by plants to convert light energy into chemical energy.'
│   │   │   │   ],
│   │   │   │   'response': 'Photosynthesis is the process by which plants convert sunlight into energy.',
│   │   │   │   'reference': 'Photosynthesis is the process by which plants and other organisms convert light energy into chemical energy.'
│   │   │   }
│   │   ],
│   │   'scores': {
│   │   │   'answer_relevancy': {
│   │   │   │   'score_rows': [
│   │   │   │   │   {'score': 0.9567410688422774},
│   │   │   │   │   {'score': 0.9262221944126402},
│   │   │   │   │   {'score': 0.8774825363469155}
│   │   │   │   ],
│   │   │   │   'aggregated_results': {'answer_relevancy': 0.9201485998672777}
│   │   │   }
│   │   }
│   },
│   eval_config={
│   │   'embedding_model': 'all-MiniLM-L6-v2',
│   │   'ragas_config': {
│   │   │   'batch_size': None,
│   │   │   'show_progress': True,
│   │   │   'raise_exceptions': True,
│   │   │   'experiment_name': None,
│   │   │   'column_map': None
│   │   }
│   }
)

In [12]:
results = client.eval.jobs.retrieve(benchmark_id=benchmark_id, job_id=job.job_id)
pprint(results)

INFO:httpx:HTTP Request: GET http://localhost:8321/v1/eval/benchmarks/ragas_demo_benchmark_20250925_192553/jobs/1/result "HTTP/1.1 200 OK"


EvaluateResponse(
│   generations=[
│   │   {
│   │   │   'user_input': 'What is the capital of France?',
│   │   │   'retrieved_contexts': ['Paris is the capital and most populous city of France.'],
│   │   │   'response': 'The capital of France is Paris.',
│   │   │   'reference': 'Paris'
│   │   },
│   │   {
│   │   │   'user_input': 'Who invented the telephone?',
│   │   │   'retrieved_contexts': [
│   │   │   │   'Alexander Graham Bell was a Scottish-American inventor who patented the first practical telephone.'
│   │   │   ],
│   │   │   'response': 'Alexander Graham Bell invented the telephone in 1876.',
│   │   │   'reference': 'Alexander Graham Bell'
│   │   },
│   │   {
│   │   │   'user_input': 'What is photosynthesis?',
│   │   │   'retrieved_contexts': [
│   │   │   │   'Photosynthesis is a process used by plants to convert light energy into chemical energy.'
│   │   │   ],
│   │   │   'response': 'Photosynthesis is the process by which plants convert sunlight into energy.',
│   │   │   'reference': 'Photosynthesis is the process by which plants and other organisms convert light energy into chemical energy.'
│   │   }
│   ],
│   scores={
│   │   'answer_relevancy': ScoringResult(
│   │   │   aggregated_results={'answer_relevancy': 0.9201485998672777},
│   │   │   score_rows=[
│   │   │   │   {'score': 0.9567410688422774},
│   │   │   │   {'score': 0.9262221944126402},
│   │   │   │   {'score': 0.8774825363469155}
│   │   │   ]
│   │   )
│   }
)